# Building an image dataset

Vamos usar o [Bing Image Search API]() da Microsoft para buscar as imagens.
Precisamos criar uma conta para pegar a chave da API

Podemos entrar no [link](https://azure.microsoft.com/en-us/try/cognitive-services/my-apis/) para pegar a chave da API

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
workdir_path = '/content/drive/My Drive/BI_evelyn/exercicios_lui_api'  # Inserir o local da pasta onde estão os arquivos de entrada (treino e teste)
os.chdir(workdir_path)

In [ ]:
from requests import exceptions
import requests
import cv2
import os

In [ ]:
query = 'beer'
output = 'beer'

Vamos pegar as primeiras 250 imagens. Porém vamos pegar 50 por request. Vamos fazer uma consulta paginada

Precisamos olhar a [documentação do endpoint](https://docs.microsoft.com/en-us/rest/api/cognitiveservices-bingsearch/bing-images-api-v7-reference) para entender como funciona


[key](https://docs.microsoft.com/pt-br/azure/cognitive-services/bing-image-search/image-sdk-python-quickstart)

In [ ]:
API_KEY = ''
MAX_RESULTS = 250
GROUP_SIZE = 50

URL = "https://api.cognitive.microsoft.com/bing/v7.0/images/search"

Ao baixar imagens da internet Python e a biblioteca requests podem lançar e exceções. Vamos definir algumas para poder tratá-las de maneira adequado quando e se ocorrerem

In [ ]:
EXCEPTIONS = set([IOError, FileNotFoundError,
                  exceptions.RequestException, exceptions.HTTPError,
                  exceptions.ConnectionError, exceptions.Timeout])

Vamos setar alguns parametros da consulta

In [ ]:
headers = {'Ocp-Apim-Subscription-Key': API_KEY}

In [ ]:
params = {'q': query, 'offset': 0, 'count': GROUP_SIZE}

Vamos fazer uma requisição para a API 

In [ ]:
search = requests.get(URL, headers=headers, params=params)
search.raise_for_status()

In [ ]:
results = search.json()

In [ ]:
estimated_num_results = min(results['totalEstimatedMatches'], MAX_RESULTS)

In [ ]:
total = 0

In [ ]:
for offset in range(0, estimated_num_results, GROUP_SIZE):
    print("Fazendo requisições para o grupo {}-{} de {}...".format(offset, offset + GROUP_SIZE,
                                                                   estimated_num_results))
    params['offset'] = offset
    search = requests.get(URL, headers=headers, params=params)
    search.raise_for_status()
    
    results = search.json()
    print("Salvando imagens para o grupo {}-{} de {}...".format(offset, offset + GROUP_SIZE,
                                                                estimated_num_results))
    for v in results['value']:
        try:
            print("Buscando: {}".format(v['contentUrl']))
            r = requests.get(v['contentUrl'], timeout=30)
            
            ext = v['contentUrl'][v['contentUrl'].rfind('.'):]
            path = os.path.sep.join([output, "{}{}".format(str(total).zfill(8), ext)])
            print(path)
            f = open(path, 'wb')
            f.write(r.content)
            f.close()
            
        except Exception as e:
            if type(e) in EXCEPTIONS:
                print('Pulando: {}'.format(v['contentUrl']))
                continue
                
        image = cv2.imread(path)
        
        if image is None:
            print('Deletando: {}'.format(path))
            #os.remove(path)
            continue
        
        total += 1